<a href="https://colab.research.google.com/github/zypchn/cAsh/blob/main/cAsh_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Part I: Data Scraping

In [ ]:
SM_urls = [
    "https://www.pricecharting.com/console/pokemon-sun-&-moon",
    "https://www.pricecharting.com/console/pokemon-guardians-rising",
    "https://www.pricecharting.com/console/pokemon-burning-shadows",
    "https://www.pricecharting.com/console/pokemon-shining-legends",
    "https://www.pricecharting.com/console/pokemon-crimson-invasion",
    "https://www.pricecharting.com/console/pokemon-ultra-prism",
    "https://www.pricecharting.com/console/pokemon-forbidden-light",
    "https://www.pricecharting.com/console/pokemon-celestial-storm",
    "https://www.pricecharting.com/console/pokemon-dragon-majesty",
    "https://www.pricecharting.com/console/pokemon-lost-thunder",
    "https://www.pricecharting.com/console/pokemon-team-up",
    "https://www.pricecharting.com/console/pokemon-detective-pikachu",
    "https://www.pricecharting.com/console/pokemon-unbroken-bonds",
    "https://www.pricecharting.com/console/pokemon-unified-minds",
    "https://www.pricecharting.com/console/pokemon-hidden-fates",
    "https://www.pricecharting.com/console/pokemon-cosmic-eclipse"
]

SWSH_urls = [
    "https://www.pricecharting.com/console/pokemon-sword-&-shield",
    "https://www.pricecharting.com/console/pokemon-rebel-clash",
    "https://www.pricecharting.com/console/pokemon-darkness-ablaze",
    "https://www.pricecharting.com/console/pokemon-champion%27s-path",
    "https://www.pricecharting.com/console/pokemon-vivid-voltage",
    "https://www.pricecharting.com/console/pokemon-shining-fates",
    "https://www.pricecharting.com/console/pokemon-battle-styles",
    "https://www.pricecharting.com/console/pokemon-chilling-reign",
    "https://www.pricecharting.com/console/pokemon-evolving-skies",
    "https://www.pricecharting.com/console/pokemon-celebrations",
    "https://www.pricecharting.com/console/pokemon-fusion-strike",
    "https://www.pricecharting.com/console/pokemon-brilliant-stars",
    "https://www.pricecharting.com/console/pokemon-astral-radiance",
    "https://www.pricecharting.com/console/pokemon-go",
    "https://www.pricecharting.com/console/pokemon-lost-origin",
    "https://www.pricecharting.com/console/pokemon-silver-tempest",
    "https://www.pricecharting.com/console/pokemon-crown-zenith"
]

SV_urls = [
    "https://www.pricecharting.com/console/pokemon-scarlet-&-violet",
    "https://www.pricecharting.com/console/pokemon-paldea-evolved",
    "https://www.pricecharting.com/console/pokemon-obsidian-flames",
    "https://www.pricecharting.com/console/pokemon-scarlet-&-violet-151",
    "https://www.pricecharting.com/console/pokemon-paradox-rift",
    "https://www.pricecharting.com/console/pokemon-paldean-fates",
    "https://www.pricecharting.com/console/pokemon-temporal-forces",
    "https://www.pricecharting.com/console/pokemon-twilight-masquerade",
    "https://www.pricecharting.com/console/pokemon-shrouded-fable",
    "https://www.pricecharting.com/console/pokemon-stellar-crown",
    "https://www.pricecharting.com/console/pokemon-surging-sparks",
    "https://www.pricecharting.com/console/pokemon-prismatic-evolutions",
    "https://www.pricecharting.com/console/pokemon-journey-together",
    "https://www.pricecharting.com/console/pokemon-destined-rivals",
    "https://www.pricecharting.com/console/pokemon-black-bolt",
    "https://www.pricecharting.com/console/pokemon-white-flare"
]

In [ ]:
import re
import pandas as pd
import json
import requests
import time
import asyncio
from bs4 import BeautifulSoup

In [ ]:
def scrape_prod_urls(url):
    time.sleep(10)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    prod_lists = soup.find("tbody")
    prod_items = prod_lists.find_all("tr")
    all_urls = [item.a["href"] for item in prod_items if item.a and item.a.get("href")]
    return all_urls

In [ ]:
def is_card(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    return str(soup.title).find("#") != -1

In [ ]:
def conditional_scrape(url):
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    prod_name = soup.title.text.split("|")[0].split("Prices")[0].strip().replace(" ", "_")
    data_script = soup.find('script', string=re.compile("VGPC.chart_data = {"))
    data = json.loads(data_script.text.split('VGPC.chart_data = ')[1].split(' VGPC.product = {')[0].split(';')[0].strip())
    return prod_name, data

In [ ]:
def transform_to_df(prod_name, data, set_name):
    df_used = pd.DataFrame(data['used'], columns = ['Date_time', 'Price'])
    df_graded = pd.DataFrame(data['graded'], columns = ['Date_time', 'Price'])
    df_prices = pd.merge(df_used, df_graded, on="Date_time")
    df_prices = df_prices.rename(columns={"Price_x": "used_price", "Price_y": "graded_price"})
    df_prices['Date_time'] = pd.to_datetime(df_prices['Date_time'], unit="ms")
    df_prices = df_prices.set_index(['Date_time'])
    df_prices["prod_name"] = prod_name
    df_prices["set_name"] = set_name
    return df_prices

In [ ]:
poke_prods = pd.DataFrame()

for set_url in SM_urls:
    set_name = set_url.split("/")[-1]
    all_urls = scrape_prod_urls(set_url)
    for url in all_urls:
        try:
            if is_card(url):
                prod_name, data = conditional_scrape(url)
                prod_df = transform_to_df(prod_name, data, set_name)
                poke_prods = pd.concat([poke_prods, prod_df])
        except Exception as e:
            pass
    print(f"data scraping for '{set_name}' has finished")

data scraping for 'pokemon-sun-&-moon' has finished
data scraping for 'pokemon-guardians-rising' has finished
data scraping for 'pokemon-burning-shadows' has finished
data scraping for 'pokemon-shining-legends' has finished
data scraping for 'pokemon-crimson-invasion' has finished
data scraping for 'pokemon-ultra-prism' has finished
data scraping for 'pokemon-forbidden-light' has finished
data scraping for 'pokemon-celestial-storm' has finished
data scraping for 'pokemon-dragon-majesty' has finished
data scraping for 'pokemon-lost-thunder' has finished
data scraping for 'pokemon-team-up' has finished
data scraping for 'pokemon-detective-pikachu' has finished
data scraping for 'pokemon-unbroken-bonds' has finished
data scraping for 'pokemon-unified-minds' has finished
data scraping for 'pokemon-hidden-fates' has finished
data scraping for 'pokemon-cosmic-eclipse' has finished


In [ ]:
poke_prods.to_csv("SM_raw.csv")

In [ ]:
SM = pd.read_csv("/content/SWSH_raw.csv")

## Part II: Explatory Data Analysis